# Extract paragraphs, combine short paragraphs

In [1]:
import re
import os
import csv
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/yuchenluo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
col_names = ["text","candidate", "term", "title", "comp"]
rows = []
for root, dirs, files, in os.walk('/Users/yuchenluo/Desktop/Measure_RadicalDiscourse'):
    for file in files:
        if file.endswith('txt'):
            speech = open(os.path.join(root, file), "r").read()
            #speech = f.read()
            #get meta data of the speech
            cand = re.sub("-speech.*", "", file)
            term = re.search("speech-(.*)-", file).group(1)[0:4]
            title = re.search("speech-(.*).txt", file).group(1)
            pars = speech.split('\n')
            pars = [i for i in pars if i] #remove empty strings
            pars = [re.sub(" \[[^()]*\]", "", par) for par in pars] #remove strings inside brackets
            # loop through all paragraphs inside each speech 
            i = 0    
            while i < (len(pars) - 1):
                n_sent = len(sent_tokenize(pars[i]))
                if n_sent >2:
                    row = [pars[i], cand, term, title, False]
                    rows.append(row)
                    i += 1
                elif i < len(pars) - 1:
                    row = [pars[i] + " " + pars[i +1], cand, term, title, True]
                    i += 2
                    if len(sent_tokenize(row[0])) < 3 and i < len(pars) - 1:
                        row_2 = [row[0] + " " + pars[i], cand, term, title, True]
                        i += 1
                        rows.append(row_2)
                    else: 
                        rows.append(row)
                else: 
                    row = [pars[i - 1] + " " + pars[i], cand, term, title, True]
                    rows.pop()
                    rows.append(row)
                            
                                
                            
                
    

AttributeError: 'NoneType' object has no attribute 'group'

In [1]:
#rows[0:10]  #check data

In [12]:
with open("paragraphs.csv", 'w') as csvfile:
    writer = csv.writer(csvfile)   
    writer.writerow(col_names)
    writer.writerows(rows)

# Repeat the process for Annenberg data

In [3]:
import pandas as pd
annen_meta = pd.read_csv("/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/annenberg_metadata.csv")

In [16]:
root = "/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/Annenberg-data" 
col_names = ["Speech_id","text","party", "term", "comp"] 
rows = [] 

for file in os.listdir('/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/Annenberg-data'): 
    if file.endswith('txt'): 
        speech = open(os.path.join(root, file), "r").read() 
        #get meta data of the speech 
        id_speech = re.search('File_(.*).txt', file).group(1) 
        party = annen_meta[annen_meta['id_speech'] == float(id_speech)]['party'].iloc[0] 
        term = annen_meta[annen_meta['id_speech'] == float(id_speech)]['year'].iloc[0] 
        pars = speech.split('\n\n') 
        Type = annen_meta[annen_meta['id_speech'] == float(id_speech)]['type'].iloc[0] 
        if Type == 'Speeches': #only keep speeches 
        # the last paragraph is not part of body text
            del pars[-1] 
            pars = [i for i in pars if i] #remove empty strings 
            pars = [re.sub(" \[[^()]*\]", "", par) for par in pars] #remove strings inside brackets 
            pars = [re.sub("\t", "",par) for par in pars]
            pars = [re.sub("\n", "",par) for par in pars] # remove \t and \n
            # loop through all paragraphs inside each speech  
            i = 0     
            while i < (len(pars) - 1): 
                n_sent = len(sent_tokenize(pars[i])) 
                if n_sent >2: 
                    row = [id_speech, pars[i], party, term, False] 
                    rows.append(row) 
                    i += 1 
                elif i < len(pars) - 1: 
                    row = [id_speech, pars[i] + " " + pars[i +1], party, term, True] 
                    i += 2 
                    if len(sent_tokenize(row[0])) < 3 and i < len(pars) - 1: 
                        row_2 = [id_speech, row[0] + " " + pars[i], party, term, True] 
                        i += 1 
                        rows.append(row_2) 
                    else:  
                        rows.append(row) 
                else:  
                    row = [id_speech, pars[i - 1] + " " + pars[i], party, term, True] 
                    rows.pop() 
                    rows.append(row) 

In [17]:
with open("paragraphs_annenberg.csv", 'w') as csvfile: 
    writer = csv.writer(csvfile)    
    writer.writerow(col_names) 
    writer.writerows(rows)

In [22]:
annen_meta[annen_meta['type'] == 'Ads']

,id_speech,type,year,party,state
0,1,Ads,1952,dem,NaN
1,2,Ads,1952,dem,NaN
2,3,Ads,1952,dem,NaN
3,4,Ads,1952,dem,NaN
4,5,Ads,1952,dem,NaN
...,...,...,...,...,...
868,869,Ads,1996,rep,NaN
869,870,Ads,1996,rep,NaN
870,871,Ads,1996,rep,NaN
871,872,Ads,1996,rep,NaN


# Combine UCSB data and Annenberg data


In [ ]:
import numpy as np

ucsb = pd.read_csv('paragraphs.csv')
annenberg = pd.read_csv('paragraphs_annenberg.csv',skipinitialspace=True)

# remove \n and \t in annenberg data 
annenberg['text'] = annenberg['text'].replace(r'\n',' ', regex=True) 
annenberg['text'] = annenberg['text'].replace(r'\t',' ', regex=True) 

In [ ]:
# create a party variable for ucsb data to match the annenberg data
dems = ['obama', 'gore-jr', 'clinton','kerry']
ucsb['party'] = ['dem' if x in dems else 'rep' for x in ucsb['candidate']] 

# match columns
ucsb_matched = ucsb[['title', 'text', 'party', 'term', 'comp']]
ucsb_matched.rename(columns = {'title':'Speech_id'}, inplace = True) 


In [ ]:
all_data = annenberg.append(ucsb_matched)

In [ ]:
all_data.to_csv('all_paragraphs.csv')

# Sample the data

In [ ]:
sample = all_data.sample(n = 1000)
sample1 = sample[0:500]
sample2 = sample[500:]

In [ ]:
sample1.to_csv('yuchen_sample.csv')
sample2.to_csv('bart_sample.csv')